In [1]:
import sys, os
sys.path.append('..')
#hls4ml_catapult_path = '/home/manuelbv/hls4ml_catapult'
#sys.path = [hls4ml_catapult_path] + sys.path

# Import datetime to get today's date
from datetime import datetime

""" Let's add our custom netsurf code """
import netsurf

""" Get netsurf path """
import os 
parent_dir = os.path.dirname(os.path.dirname(os.path.abspath(netsurf.__file__)))

Adding /Users/mbvalentin/scripts/netsurf to sys.path
[INFO] - Added qkeras to sys.path from /Users/mbvalentin/scripts/netsurf/qkeras
[INFO] - Added pergamos to sys.path from /Users/mbvalentin/scripts/netsurf/pergamos
[INFO] - Loaded theme: default
                   ▚ ▞▛ ▝   ▘   ▞  
   𐌍 ↠⌾↞ 𐌃 𐌖 𐌔     ▝▗▞ ▖ ▙ ▞   ▜▛  
                   ▜  ▛ ▞ ▙  ▜     

Logging to file: /Users/mbvalentin/.nodus/nodus_20250415_233942.log
Found config file: /Users/mbvalentin/.netsurf/config


-- Date: 15/Apr/2025
╭───────┬─────────────╮
╰ INFO ─┤ 23:39:42.26 │ - Nodus initialized
        │ 23:39:42.26 │ - Nodus version: 0.1.0
        │ 23:39:42.26 │ - Nodus imported
        │ 23:39:42.26 │ - Jobs imported
        │ 23:39:42.26 │ - JobManager imported
        │ 23:39:42.26 │ - Nodus ready to use
        │ 23:39:42.41 │ - Created jobs table in NodusDB instance 'netsurf_db'
        │ 23:39:42.41 │ - Created job_dependencies table in NodusDB instance 'netsurf_db'
        │ 23:39:42.41 │ - Added NodusDB instance 'netsurf_db' linked to database 'netsurf_db'


In [2]:
import pergamos as pg

benchmark_name = 'keyword_spotting'
# Set variables
qscheme = "q<6,2,1>"
pruning = 0.125
prerank = True


""" First of all, let's define a quantization Scheme """
Q = netsurf.QuantizationScheme(qscheme)
print(Q)

# Set filename
benchmarks_dir = os.path.join(parent_dir, 'benchmarks')
datasets_dir = os.path.join(parent_dir, 'datasets')

filename = f"7_HLS_{Q._scheme_str.no_special_chars()}_bmark_{benchmark_name}_prune{str(pruning).replace('.','_')}.html"
print(filename)
doc = pg.Document(filename, theme="default")
doc.required_scripts.add('mathjax')


""" Add a title to the document """
doc.append(pg.Markdown(f"""# Benchmarks Quantization Assertion
> Author: Manuel B Valentin

> Creation date: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}

> Project: netsurf

> Used packages: netsurf, tensorflow, numpy, matplotlib, pergamos
        
"""))

# Add a tab container
tabs = pg.TabbedContainer({'Motivation': [], 
                        'Pre-training analysis': [],
                        'Training': [],
                            'Post-Training': [],
                        'BER Injection': [],
                        'Conclusions': []})

# Get individual tabs
tabmotivation = tabs['Motivation']
tabpretraining = tabs['Pre-training analysis']
tabtraining = tabs['Training']
tabposttraining = tabs['Post-Training']
tabber = tabs['BER Injection']
tabconclusions = tabs['Conclusions']

# Add to documnt
doc.append(tabs)

# Add a markdown description of what we want to achieve with this report in the first tab
md = r"""
## 1. Loss Taylor expansion

Given a loss function $\mathcal{L}(w)$, where $w$ is the vector of all weights in the network, the Taylor expansion around some point $w_0$ (say, the trained weights) for a small perturbation $\Delta w$ is:

$$\mathcal{L}(w_0 + \Delta w) \approx \mathcal{L}(w_0) + \nabla \mathcal{L}(w_0)^T \Delta w + \frac{1}{2} \Delta w^T H \Delta w$$

Where:

* $\nabla \mathcal{L}(w_0)$ is the gradient vector of the loss at w_0
* $H$ is the Hessian matrix, i.e. $H = \nabla^2 \mathcal{L}(w_0)$

---

## 2. If the model is trained??

If the model has been well trained, then:

$\nabla \mathcal{L}(w_0) \approx 0$

Because you're sitting near a (local) minimum.

This removes the linear term:
$\mathcal{L}(w_0 + \Delta w) - \mathcal{L}(w_0) \approx \frac{1}{2} \Delta w^T H \Delta w$

So the change in loss caused by a perturbation $\Delta w$ is approximately:
$\Delta \mathcal{L} \approx \frac{1}{2} \Delta w^T H \Delta w$

---

## 3. Interpretation for bit flips

A bit flip in the quantized weights causes a small but structured change in the weights:

* Say, flipping the 3rd bit in weight $w_i$ causes it to change by $\delta_i$, so:

$\Delta w = \begin{bmatrix}
0 \\ \cdots \\ \delta_i \\ \cdots \\ 0
\end{bmatrix}$

Then the loss increase is (approximately):
$\Delta \mathcal{L} \approx \frac{1}{2} \delta_i^2 H_{ii}$

If multiple bits are flipped across weights, you sum their pairwise interactions via H, including off-diagonal terms (if not ignored).

--- 

## 4. Implications for ranking

This approximation motivates ranking bit positions (or weights) by:

* $\delta^2 \cdot H_{ii}$: bit-flip magnitude times curvature
* This is the FKeras method: estimates $H_{ii}$ and ranks accordingly
* You could generalize it to your method:
$\text{Impact} \cdot H$, not just gradients

---

## 5. When does this approximation hold?

?? Works well when:

* Bit-flip magnitude is small (i.e., local region)
* Model is near a minimum
* Hessian is stable (not exploding)

?? Fails when:

* Model isn??t trained well (gradient is large)
* Loss surface is highly non-quadratic

---

## Summary

The formula:
$\Delta \mathcal{L} \approx \frac{1}{2} \Delta \mathbf{w}^T H \Delta \mathbf{w}$

tells us how bit-flips propagate into loss increases, and explains why the Hessian is so powerful for ranking robustness. 
It encodes:

* How impactful a perturbation is (via $\delta$)
* How sensitive the loss is locally (via $H$)

"""

# Create markdown
md = pg.Markdown(md)

# Add to the first tab
tabmotivation.append(md)

""" Add quantization container to doc report """
tabpretraining.extend(Q.html())

""" Save doc to file (we save after adding each element) """
doc.save(filename)

# Let's create a container for all benchmarks 
benchmark_ct = pg.CollapsibleContainer("🧺 Benchmarks", layout='vertical')

# And another one for tabtraining
benchmark_sessions_ct = pg.CollapsibleContainer("🧺 Benchmarks", layout='vertical')

# And another one for tabposttraining
benchmark_posttraining_ct = pg.CollapsibleContainer("🧺 Benchmarks", layout='vertical')

# And yet another for "BER Injection"
benchmark_ber_ct = pg.CollapsibleContainer("🧪 Experiments", layout='vertical')

""" Add to documnt """
tabpretraining.append(benchmark_ct)

""" Add """
tabtraining.append(benchmark_sessions_ct)

""" Add """
tabposttraining.append(benchmark_posttraining_ct)

""" Add """
tabber.append(benchmark_ber_ct)

# Define benchmarks to analyze
#benchmarks = ['dummy', 'mnist_hls4ml', 'autompg', 'smartpixel_small', 'smartpixel_large',
#              'cifar10', 'mnist_lenet5', 'ECONT_AE'
# 'cifar100', 'svhn', 'fashion_mnist', 'imdb', 'reuters', 'boston_housing']
# TODO: Fix visualization/contrast for cifar10
# TODO: mnist_lenet5 seems to be working (good accuracy), but I'm not too happy about the alphas/betas. Some layers still have a big portion outside of the valid interval

config_per_methods = netsurf.config.config_per_method
protection_range = (0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4) #netsurf.config.DEFAULT_PROTECTION
ber_range = (0.001, 0.005, 0.01, 0.05, 0.1, 0.15) #netsurf.config.DEFAULT_BER

#methods = ['qpolar', 'qpolargrad', 'bitwise_msb', 'random', 'hirescam_norm', 
#           'hiresdelta', 'hessian', 'hessiandelta', 'weight_abs_value']
methods = ['laplace', 'hessian', 'fisher', 'bitwise_msb', 'random', 'grad_norm', 'graddelta', 'weight_abs_value'] # 'qpolar', 'qpolargrad', 
methods = methods

# Create benchmark object 
bmk = netsurf.get_benchmark(benchmark_name, Q,  benchmarks_dir = benchmarks_dir,
                            datasets_dir = datasets_dir, pruning = pruning,
                            load_weights = False)

# Add benchmark html to container (this includes model + dataset htmls)
# (run before training the model...)
benchmark_ct.append(bmk.html())

# Now let's prepare the data
nsample_mod = 48 if 'ECON' in bmk.name else -1
XYTrain = netsurf.utils.prepare_data(bmk, subset = 'train', nsample_mod = nsample_mod)

# # Initialize the uncertainty profiler (pre-training)
# pre_robustness_sgn_path = os.path.join(bmk.model_dir, 'uncertainty_signatures', f'{benchmark_name}.pretraining.netsurf.sgn')
# try:
#     pre_robustness_sgn = netsurf.UncertaintyProfiler.profile(bmk.model, XYTrain, bmk.model.loss, 
#                                                             batch_size = 2000, filepath = pre_robustness_sgn_path,
#                                                             verbose = True)
#     # Save 
#     pre_robustness_sgn.save_to_file(pre_robustness_sgn_path)
# except:
#     pass

# Add profile to tabpretraining
# benchmark_ct.append(pre_robustness_sgn.html())

# Now we can reload the weights (After the profiling)
bmk.load_weights(verbose = True)

# TRAINING - SESSION
# Try to get a session (if not, train)
sess = netsurf.get_training_session(bmk, show_plots = False, plot = True)


🧮 <QuantizationScheme(q<6,2,1>)> obj @ (0x16306d870):
    Total number of bits (m): 6
    Integer bits         (n): 2
    Float bits           (f): 3
    Signed
    Range: (-4, 3.875)
    Min step: 0.125
    Format: Sxx.xxx

7_HLS_q6_2_1_bmark_keyword_spotting_prune0_125.html


╭───────┼─────────────┤
╰ BMK ──┤ 23:39:43.41 │ - Initializing benchmark object keyword_spotting
        │ 23:39:43.41 │ - Loss sparse_categorical_crossentropy found in tf.keras.losses with definition <function sparse_categorical_crossentropy at 0x1456de7a0>.
        │ 23:39:43.41 │ - Adding custom metric sparse_categorical_accuracy with definition SparseCategoricalAccuracy(name=sparse_categorical_accuracy,dtype=float32).
        │ 23:39:45.53 │ - Benchmark object keyword_spotting initialized
╭───────┼─────────────┤
╰ PLOT ─┤ 23:39:54.18 │ - Variable qq_conv2d_1/kernel:0 has sparsity 0.00%
        │ 23:39:54.18 │ - Variable qq_conv2d_4/kernel:0 has sparsity 0.00%
        │ 23:39:54.19 │ - Variable qq_conv2d_3/kernel:0 has sparsity 0.00%
        │ 23:39:54.19 │ - Variable qq_conv2d_2/kernel:0 has sparsity 0.00%
        │ 23:39:54.20 │ - Variable qq_dense/kernel:0 has sparsity 0.00%
        │ 23:39:54.20 │ - Variable qq_conv2d/kernel:0 has sparsity 0.00%
        │ 23:39:54.20 │ - Variabl

[WARN] - HTML not implemented for keyword_spotting


╭───────┼─────────────┤
╰ PLT ──┤ 23:40:01.84 │ - Plotting histogram for activation qq_conv2d
        │ 23:40:01.91 │ - Plotting histogram for activation qq_apply_alpha
        │ 23:40:01.98 │ - Plotting histogram for activation qq_batch_normalization
        │ 23:40:02.06 │ - Plotting histogram for activation qq_activation
        │ 23:40:02.11 │ - Plotting histogram for activation qq_dropout
        │ 23:40:02.16 │ - Plotting histogram for activation qq_depthwise_conv2d
        │ 23:40:02.23 │ - Plotting histogram for activation qq_apply_alpha_1
        │ 23:40:02.31 │ - Plotting histogram for activation qq_batch_normalization_1
        │ 23:40:02.38 │ - Plotting histogram for activation qq_activation_1
        │ 23:40:02.43 │ - Plotting histogram for activation qq_conv2d_1
        │ 23:40:02.50 │ - Plotting histogram for activation qq_apply_alpha_2
        │ 23:40:02.57 │ - Plotting histogram for activation qq_batch_normalization_2
        │ 23:40:02.65 │ - Plotting histogram for ac

KeyboardInterrupt: 

In [ ]:
bmk.model.summary()

In [ ]:

# Create a container for this bmk in tabtraining
bmk_sess_ct = pg.CollapsibleContainer(benchmark_name, layout='vertical')

# Add session to tabtraining
bmk_sess_ct.append(sess.html())

# And add to benchmarks in tabtraining
benchmark_sessions_ct.append(bmk_sess_ct)

# Add benchmark again to post-training to check how the weights changed
#benchmark_posttraining_ct.append(bmk.html())

# Initialize the uncertainty profiler (post-training)
# try:
#     post_robustness_sgn_path = os.path.join(bmk.model_dir, 'uncertainty_signatures', f'{benchmark_name}.posttraining.netsurf.sgn')
#     post_robustness_sgn = netsurf.UncertaintyProfiler.profile(bmk.model, XYTrain, bmk.model.loss, 
#                                     batch_size = 2000, verbose = True, filepath = post_robustness_sgn_path)

#     # Save 
#     post_robustness_sgn.save_to_file(post_robustness_sgn_path)
# except:
#     pass

# Add profile to tabposttraining
# benchmark_posttraining_ct.append(post_robustness_sgn.html())

# Let's compare the divergence between the profiles
# div_profile = netsurf.ProfileDivergence.from_signatures(pre_robustness_sgn, post_robustness_sgn)

#div_profile.plot_divergence_summary()
#div_profile.plot_advanced_divergence_summary()

""" Save doc to file (we save after adding each element) """
doc.save(filename)

# Create a container for this benchmark in "BER Injection"
bmk_ber_ct = pg.CollapsibleContainer(benchmark_name, layout='vertical')

# Add to tabber
benchmark_ber_ct.append(bmk_ber_ct)

# Create another container to store the individual results of the experiments
exp_individual_ct = pg.CollapsibleContainer("🎁 Individual results", layout='vertical')
bmk_ber_ct.append(exp_individual_ct)

# Create our ranker profiler (empty)
rankers_bucket = netsurf.RankingComparator()

# Loop thru methods
exps = {}
for method_alias in methods:
    #########################################################################
    # 0. Create the directory for the experiment (ranking, results, etc.)
    #########################################################################
    method = config_per_methods[method_alias]['method']
    method_dir = os.path.join(bmk.experiments_dir, method)

    #########################################################################
    # 1. Create the ranker
    #########################################################################
    rkr = rankers_bucket.build_ranker(method, Q, config = config_per_methods[method], 
                                    path = method_dir,
                                    is_baseline = (method == 'qpolargrad'))

    # Get the exp dir from the ranker (with the hash)
    exp_dir = rkr.path

    #################################################################
    # 1. Perform ranking according to method
    #################################################################
    # Rank weights 
    ranking = rkr.rank(bmk.model, *XYTrain, verbose = True)
    # Save rank to csv file 
    ranking.save(overwrite = False)

    #################################################################
    # 2. Create experiment object
    #################################################################
    exp = netsurf.Experiment(bmk, ranking, num_reps = 100, 
                            ber_range = ber_range, 
                            protection_range = protection_range, 
                            path = exp_dir,
                            verbose = True)

    # Print experiment info 
    print(exp)

    #################################################################
    # 3. Run experiment with given ranking and for whatever 
    #       range of protection and rad 
    #################################################################
    #batch_size = 1000,
    exp.run_experiment(bmk, XYTrain,
                    batch_size = None,
                    ber_range = ber_range, 
                    protection_range = protection_range, 
                    rerun = False)
    
    # Add experiment to container
    #exp_individual_ct.append(exp.html())

    # Save experiment object
    #exp.save()

    # Add to dict
    exps[method] = exp

    """ Save doc to file (we save after adding each element) """
    doc.save(filename)

""" Now perform the comparison of rankers """
#df = rankers_bucket.compare_rankers(granularity = 0.01, bootstrap = False)

# Add comparison to container
#bmk_ber_ct.append(rankers_bucket.html())

# Save doc 
doc.save(filename)

# # Save the rankers_bucket
# rankers_comparison_filepath = os.path.join(bmk.experiments_dir, 'ranking_comparison.csv')
# rankers_bucket.save_to_csv(rankers_comparison_filepath)

# """ Create Experiment Comparator """
# exp_comp = netsurf.ExperimentComparator(list(exps.values()))
#df = ExperimentComparator.compute_ranking_distribution(res)